<a href="https://colab.research.google.com/github/Pravanop/ml4Sci_eval/blob/main/Task_1_(Common).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import h5py
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split

## Reading data

### Electron Dataset

In [3]:
data_elec = h5py.File('/content/drive/MyDrive/GSOC/Application/ML4Sci/Task 1/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5','r')
data_elec.keys()

<KeysViewHDF5 ['X', 'y']>

In [4]:
electron_X = data_elec['X']
electron_Y = data_elec['y']

In [5]:
#checking shapes
print(electron_X.shape)
print(electron_Y.shape)

(249000, 32, 32, 2)
(249000,)


### Photon Dataset

In [6]:
data_pho = h5py.File('/content/drive/MyDrive/GSOC/Application/ML4Sci/Task 1/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5','r')

In [7]:
photon_X = data_pho['X']
photon_Y = data_pho['y']

In [8]:
print(photon_X.shape)
print(photon_Y.shape)

(249000, 32, 32, 2)
(249000,)


In [9]:
#checking assignment of labels
print(electron_Y[42])
print(photon_Y[42])

1.0
0.0


In [10]:
#joining both datasets
X = np.concatenate((np.array(photon_X), np.array(electron_X)))
Y = np.concatenate((np.array(photon_Y), np.array(electron_Y)))

In [11]:
#checking shapes
print(X.shape)
print(Y.shape)

(498000, 32, 32, 2)
(498000,)


## Building model

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42) #test split of 0.25

In [ ]:
X_train.shape

(373500, 32, 32, 2)

In [ ]:
y_train, y_test = y_train.reshape(y_train.shape[0],1), y_test.reshape(y_test.shape[0],1)

### Tensorflow version

### CNN

In [ ]:
from keras.layers import LSTM
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Flatten, Dropout, Activation, Lambda, Permute, Reshape

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, GlobalAveragePooling2D

In [ ]:
#create model
model = Sequential()
#add model layers
model.add(Conv2D(128, kernel_size= (3,3), activation= 'relu', input_shape=(32,32,2)))
model.add(Conv2D(64, kernel_size= (3,3), activation= 'relu'))
model.add(MaxPooling2D((2, 2), padding = 'valid'))
model.add(Conv2D(32, kernel_size= (3, 3), activation= 'relu'))
model.add(Conv2D(32, kernel_size= (3, 3), activation= 'relu'))
model.add(Conv2D(32, kernel_size= (3, 3), activation= 'relu'))
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(1, activation= 'sigmoid'))

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs = 10,batch_size = 256)

Epoch 1/10
1459/1459 [==============================] - 80s 32ms/step - loss: 0.6398 - accuracy: 0.6301 - val_loss: 0.5801 - val_accuracy: 0.7013
Epoch 2/10
1459/1459 [==============================] - 45s 31ms/step - loss: 0.5775 - accuracy: 0.7052 - val_loss: 0.5699 - val_accuracy: 0.7108
Epoch 3/10
1459/1459 [==============================] - 45s 31ms/step - loss: 0.5632 - accuracy: 0.7171 - val_loss: 0.5585 - val_accuracy: 0.7200
Epoch 4/10
1459/1459 [==============================] - 45s 31ms/step - loss: 0.5542 - accuracy: 0.7240 - val_loss: 0.5513 - val_accuracy: 0.7255
Epoch 5/10
1459/1459 [==============================] - 45s 31ms/step - loss: 0.5492 - accuracy: 0.7276 - val_loss: 0.5526 - val_accuracy: 0.7254
Epoch 6/10
1459/1459 [==============================] - 45s 31ms/step - loss: 0.5460 - accuracy: 0.7298 - val_loss: 0.5504 - val_accuracy: 0.7283
Epoch 7/10
1459/1459 [==============================] - 45s 31ms/step - loss: 0.5417 - accuracy: 0.7329 - val_loss: 0.5455 -

### LSTM

In [ ]:
X_train, X_test = X_train.reshape(X_train.shape[0], 1, X_train.shape[1]*X_train.shape[2]*X_train.shape[3]), X_test.reshape(X_test.shape[0], 1, X_test.shape[1]*X_test.shape[2]*X_test.shape[3])

In [ ]:
model = Sequential()

input_shape=(X_train.shape[1],X_train.shape[2])

model.add(LSTM(units= 64,return_sequences=True,input_shape=input_shape))
model.add(Dropout(0.2))

model.add(LSTM(units=32,return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=16,return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=16,input_shape=input_shape)) 
model.add(Dropout(0.2))

model.add(Dense(units=1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics = 'accuracy')

In [ ]:
model.fit(X_train,y_train, validation_data = (X_test, y_test), epochs= 10,batch_size= 128)

Epoch 1/10
2918/2918 [==============================] - 65s 10ms/step - loss: 0.6642 - accuracy: 0.5947 - val_loss: 0.6383 - val_accuracy: 0.6387
Epoch 2/10
2918/2918 [==============================] - 27s 9ms/step - loss: 0.6374 - accuracy: 0.6412 - val_loss: 0.6315 - val_accuracy: 0.6471
Epoch 3/10
2918/2918 [==============================] - 27s 9ms/step - loss: 0.6297 - accuracy: 0.6502 - val_loss: 0.6246 - val_accuracy: 0.6556
Epoch 4/10
2918/2918 [==============================] - 28s 9ms/step - loss: 0.6195 - accuracy: 0.6625 - val_loss: 0.6173 - val_accuracy: 0.6678
Epoch 5/10
2918/2918 [==============================] - 28s 10ms/step - loss: 0.6098 - accuracy: 0.6735 - val_loss: 0.6115 - val_accuracy: 0.6731
Epoch 6/10
2918/2918 [==============================] - 28s 9ms/step - loss: 0.6023 - accuracy: 0.6816 - val_loss: 0.6068 - val_accuracy: 0.6787
Epoch 7/10
2918/2918 [==============================] - 27s 9ms/step - loss: 0.5925 - accuracy: 0.6931 - val_loss: 0.6019 - val_

### VGGNet

In [ ]:
vgg = tf.keras.applications.VGG16(include_top = False, input_shape = (X_train.shape[1],X_train.shape[2],X_train.shape[3]), classes = 2, classifier_activation= 'sigmoid', weights = None )

In [ ]:
global_avg = GlobalAveragePooling2D()
pred_layer = Dense(1, activation='sigmoid')
net = Sequential([vgg,global_avg,pred_layer])

In [ ]:
net.compile(optimizer='adam', loss='binary_crossentropy', metrics = 'accuracy')
net.fit(X_train,y_train, validation_data = (X_test, y_test), epochs= 10,batch_size= 256)

## Pytorch

CNN was best so trying that out in pytorch

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import torch.optim as optim
from torch.optim import lr_scheduler

In [13]:
from torchvision import datasets, transforms 
from torch.utils import data  
dataset = [X,Y]
loader = data.DataLoader(dataset, batch_size = 8, shuffle = True)

In [14]:
class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(1, 4, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining another 2D convolution layer
            Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
        )

        self.linear_layers = Sequential(
            Linear(4 * 7 * 7, 10)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x


In [17]:
model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.07)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=196, out_features=10, bias=True)
  )
)


In [22]:
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            current_loss = 0.0
            current_corrects = 0

            print('Iterating through data...')

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.permute(0, 3, 1, 2)
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs.float())
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels.long())

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                current_loss += loss.item() * inputs.size(0)
                current_corrects += torch.sum(preds == labels.data)

            epoch_loss = current_loss / dataset_sizes[phase]
            epoch_acc = current_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # Make a copy of the model if the accuracy on the validation set has improved
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_since = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_since // 60, time_since % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model